In [439]:
import torch

import numpy as np
import time

from buffer import ReplayBuffer
import gymnasium as gym        

device = torch.device('mps')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [440]:
class channelFirst(gym.ObservationWrapper):
    def __init__(self, env: gym.Env):
        gym.ObservationWrapper.__init__(self, env)
        
    def observation(self, observation):
        return observation.T

env = gym.make('ALE/Breakout-v5', render_mode="rgb_array")
env = gym.wrappers.ResizeObservation(env, shape=(64,64))
env = gym.wrappers.NormalizeObservation(env)
env = channelFirst(env)
obs, _ = env.reset()

In [441]:
buffer = ReplayBuffer(50, env, False)
print(len(buffer))


-----------initialized memory----------              

obs_buffer_shape: (50, 3, 64, 64)
actions_buffer_shape: (50, 1)
rewards_buffer_shape: (50, 1)
nxt_obs_buffer_shape: (50, 3, 64, 64)
dones_buffer_shape: (50, 1)

----------------------------------------
              
0


In [442]:
from tqdm import tqdm

for i in tqdm(range(60)):
    action = env.action_space.sample()
    next_obs, reward, termination, truncation, _ = env.step(action)
    
    buffer.add(obs, action, reward, next_obs, termination | truncation)
    obs = next_obs

100%|██████████| 60/60 [00:00<00:00, 957.88it/s]


In [443]:
print(len(buffer))
print(buffer.full)

10
True


In [444]:
obs, actions, rewards, next_obs, dones = buffer.sample(50,5, device)
print(obs.shape)
print(actions.shape)
print(rewards.shape)
print(next_obs.shape)
print(dones.shape)

torch.Size([50, 5, 3, 64, 64])
torch.Size([50, 5, 1])
torch.Size([50, 5, 1])
torch.Size([50, 5, 3, 64, 64])
torch.Size([50, 5, 1])
